<a href="https://colab.research.google.com/github/rdsiese/MANDES/blob/main/5_Spread_options.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Spread Options

Consider two assets whose prices can be modeled (under a risk-neutral probability) as:

$S_{1t} = S_{10} e^{(r - \sigma_1^2/2) + \sigma_1\sqrt{t}z_{1t}}$

$S_{2t} = S_{20} e^{(r - \sigma_2^2/2) + \sigma_2\sqrt{t}z_{2t}}$

where $z_1$ and $z_2$ are two correlated standard normal random variables. Let $u$ be another standard normal distribution, independent of $z_1$, and define:

$z_2 = \rho z_1 + \sqrt{1-\rho^2} u$,

where $\rho$ is a parameter that may take any value in $[-1,1]$. The correlation between $z_1$ and $z_2$ is therefore $\text{corr}(z_1,z_2) = \rho$.

In [ ]:
import numpy as np

S10=100; S20=100; K=0; r=0.05; sigma1=0.3; sigma2=0.2; T=1; n=10000

The payoff of a **spread option** is given by $\max(S_T^2 - S_T^1 - K, 0)$, where $K$ is the strike and $T$ is maturity.

In the code below we define a function $\texttt{Spread\_Option}$ that returns the option price. We follow these steps:
> (1) Create containers to store the asset prices and the payoff ($\texttt{S1, S2, payoff}$). <br/>(2) Generate the standard normals $z_1$ and $u$. <br/>(3) Build $z_2$ in such a way that $\text{corr}(z_1,z_2) = \rho$. <br/>(4) Compute the values at maturity for both assets, $S_{1T}$ and $S_{2T}$. <br/>(5) Compute the mean payoff and discount it to present value.

In [ ]:
def Spread_Option(S10, S20, K, r, sigma1, sigma2, rho, T, n):       # Using Regular Sampling (RS)
    S1=np.zeros(n)
    S2=np.zeros(n)
    payoff=np.zeros(n)

    z1= np.random.normal(0,1,n)
    u = np.random.normal(0,1,n)
    z2= rho*z1 + np.sqrt(1-rho**2)*u

    S1=S10*np.exp((r-0.5*sigma1**2)*T + sigma1*np.sqrt(T)*z1)   #Price at T
    S2=S20*np.exp((r-0.5*sigma2**2)*T + sigma2*np.sqrt(T)*z2)   #Price at T

    payoff=np.maximum(S1-S2-K, 0)
    return np.exp(-r*T)*np.mean(payoff)

For different values of the correlation coefficient $\rho$ we use the function above ($\texttt{Spread\_Option}$) to calculate the spread option price.

In [ ]:
for rho in np.arange(0,1,0.1).round(1):
    print(f'corr: {rho:<7.1f} price: {Spread_Option(S10,S20,K,r,sigma1,sigma2,rho,T,n):.5f}')

### The Spread Option Price with Latin Hypercube Sampling (LHS)

To compute the option price with LHS we need to import new packages.

In [ ]:
%pip install pyDOE2
from pyDOE2 import lhs
from scipy.stats import norm

The function $\texttt{LH\_Spread}$ below computes the price of the spread option using LHS.

In [ ]:
def LH_Spread(S10, S20, K, r, sigma1, sigma2, rho, T, n):       # Using latin hypercube sampling (LHS)
    S1=np.zeros(n)
    S2=np.zeros(n)
    payoff=np.zeros(n)

    lhs_samples = lhs(1, samples=n)
    z1 = norm.ppf(lhs_samples)
    lhs_samples = lhs(1, samples=n)
    u = norm.ppf(lhs_samples)
    z2= rho*z1 + np.sqrt(1-rho**2)*u

    S1=S10*np.exp((r-0.5*sigma1**2)*T + sigma1*np.sqrt(T)*z1)   #Price at T
    S2=S20*np.exp((r-0.5*sigma2**2)*T + sigma2*np.sqrt(T)*z2)   #Price at T

    payoff=np.maximum(S1-S2-K, 0)
    return np.exp(-r*T)*np.mean(payoff)

for rho in np.arange(0,1,0.1).round(1):
    print(f'corr: {rho:<7.1f} price: {LH_Spread(S10,S20,K,r,sigma1,sigma2,rho,T,n):.5f}')